In [1]:
! pip install datasets transformers

     |████████████████████████████████| 311 kB 24.4 MB/s 
     |████████████████████████████████| 3.4 MB 71.8 MB/s 
     |████████████████████████████████| 133 kB 58.4 MB/s 
     |████████████████████████████████| 67 kB 699 kB/s 
     |████████████████████████████████| 243 kB 70.9 MB/s 
     |████████████████████████████████| 1.1 MB 59.6 MB/s 
     |████████████████████████████████| 3.3 MB 59.3 MB/s 
     |████████████████████████████████| 895 kB 64.9 MB/s 
     |████████████████████████████████| 596 kB 70.0 MB/s 
     |████████████████████████████████| 271 kB 76.4 MB/s 
     |████████████████████████████████| 144 kB 79.0 MB/s 
     |████████████████████████████████| 160 kB 72.4 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [24]:
from huggingface_hub import notebook_login

notebook_login()

Login successful
Your token has been saved to /root/.huggingface/token
Authenticated through git-credential store but this isn't the helper defined on your machine.
You might have to re-authenticate when pushing to the Hugging Face Hub. Run the following command in your terminal in case you want to set this credential helper as the default

git config --global credential.helper store


In [3]:
!apt install git-lfs

Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following NEW packages will be installed:
  git-lfs
0 upgraded, 1 newly installed, 0 to remove and 37 not upgraded.
Need to get 2,129 kB of archives.
After this operation, 7,662 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu bionic/universe amd64 git-lfs amd64 2.3.4-1 [2,129 kB]
Fetched 2,129 kB in 1s (2,872 kB/s)
Selecting previously unselected package git-lfs.
(Reading database ... 155229 files and directories currently installed.)
Preparing to unpack .../git-lfs_2.3.4-1_amd64.deb ...
Unpacking git-lfs (2.3.4-1) ...
Setting up git-lfs (2.3.4-1) ...
Processing triggers for man-db (2.8.3-2ubuntu0.1) ...


In [4]:
import transformers

print(transformers.__version__)

4.15.0


In [75]:
task = "sst2"
model_checkpoint = "siebert/sentiment-roberta-large-english"
batch_size = 16

In [57]:
from datasets import load_dataset, load_metric

In [58]:
actual_task = "mnli" if task == "mnli-mm" else task
dataset = load_dataset("glue", actual_task)
metric = load_metric('glue', actual_task)

Reusing dataset glue (/root/.cache/huggingface/datasets/glue/sst2/1.0.0/dacbe3125aa31d7f70367a07a8a9e72a5a0bfeb5fc42e75c9db75b96da6053ad)


  0%|          | 0/3 [00:00<?, ?it/s]

In [59]:
dataset["train"][2]

{'idx': 2,
 'label': 1,
 'sentence': 'that loves its characters and communicates something rather beautiful about human nature '}

In [60]:
import datasets
import random
import pandas as pd
from IPython.display import display, HTML

def show_random_elements(dataset, num_examples=10):
    assert num_examples <= len(dataset), "Can't pick more elements than there are in the dataset."
    picks = []
    for _ in range(num_examples):
        pick = random.randint(0, len(dataset)-1)
        while pick in picks:
            pick = random.randint(0, len(dataset)-1)
        picks.append(pick)
    
    df = pd.DataFrame(dataset[picks])
    for column, typ in dataset.features.items():
        if isinstance(typ, datasets.ClassLabel):
            df[column] = df[column].transform(lambda i: typ.names[i])
    display(HTML(df.to_html()))


In [61]:
show_random_elements(dataset["train"])

,sentence,label,idx
0,a great movie,positive,14592
1,"entertaining , if somewhat standardized , action",positive,3278
2,"even when there are lulls , the emotions seem authentic ,",positive,36048
3,a larger-than-life figure,positive,32098
4,"nothing special and , until the final act , nothing",negative,29256
5,a journey back to your childhood,positive,18289
6,a film that manages to find greatness in the hue of its drastic iconography,positive,13434
7,much funnier film,positive,11395
8,relies on more than special effects,positive,55302
9,"the music makes a nice album ,",positive,4165


In [62]:
metric

Metric(name: "glue", features: {'predictions': Value(dtype='int64', id=None), 'references': Value(dtype='int64', id=None)}, usage: """
Compute GLUE evaluation metric associated to each GLUE dataset.
Args:
    predictions: list of predictions to score.
        Each translation should be tokenized into a list of tokens.
    references: list of lists of references for each translation.
        Each reference should be tokenized into a list of tokens.
Returns: depending on the GLUE subset, one or several of:
    "accuracy": Accuracy
    "f1": F1 score
    "pearson": Pearson Correlation
    "spearmanr": Spearman Correlation
    "matthews_correlation": Matthew Correlation
Examples:

    >>> glue_metric = datasets.load_metric('glue', 'sst2')  # 'sst2' or any of ["mnli", "mnli_mismatched", "mnli_matched", "qnli", "rte", "wnli", "hans"]
    >>> references = [0, 1]
    >>> predictions = [0, 1]
    >>> results = glue_metric.compute(predictions=predictions, references=references)
    >>> print(res

In [63]:
import numpy as np

fake_preds = np.random.randint(0, 2, size=(64,))
fake_labels = np.random.randint(0, 2, size=(64,))
metric.compute(predictions=fake_preds, references=fake_labels)

{'accuracy': 0.5625}

In [64]:
from transformers import AutoTokenizer
    
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint, use_fast=True)

https://huggingface.co/siebert/sentiment-roberta-large-english/resolve/main/tokenizer_config.json not found in cache or force_download set to True, downloading to /root/.cache/huggingface/transformers/tmpjjf3vuhb


Downloading:   0%|          | 0.00/256 [00:00<?, ?B/s]

storing https://huggingface.co/siebert/sentiment-roberta-large-english/resolve/main/tokenizer_config.json in cache at /root/.cache/huggingface/transformers/9f284be68d0cfa5298eea908fc7f51cc7e0b01c7aaf2eba99c70d938b169bfe8.ba4a40df74471cdc82cd580af48bbbcfd25e9095a9d4bb296f711f3af7e2619e
creating metadata file for /root/.cache/huggingface/transformers/9f284be68d0cfa5298eea908fc7f51cc7e0b01c7aaf2eba99c70d938b169bfe8.ba4a40df74471cdc82cd580af48bbbcfd25e9095a9d4bb296f711f3af7e2619e
https://huggingface.co/siebert/sentiment-roberta-large-english/resolve/main/config.json not found in cache or force_download set to True, downloading to /root/.cache/huggingface/transformers/tmp6ty4bh0v


Downloading:   0%|          | 0.00/687 [00:00<?, ?B/s]

storing https://huggingface.co/siebert/sentiment-roberta-large-english/resolve/main/config.json in cache at /root/.cache/huggingface/transformers/228e83e1ade2247aebc5f0725e330fa58dedee3d9eec36c9249f25084a946130.1aece0680a18a95d51d6e1a5f83631412da37b87db65380c52052161354505ba
creating metadata file for /root/.cache/huggingface/transformers/228e83e1ade2247aebc5f0725e330fa58dedee3d9eec36c9249f25084a946130.1aece0680a18a95d51d6e1a5f83631412da37b87db65380c52052161354505ba
loading configuration file https://huggingface.co/siebert/sentiment-roberta-large-english/resolve/main/config.json from cache at /root/.cache/huggingface/transformers/228e83e1ade2247aebc5f0725e330fa58dedee3d9eec36c9249f25084a946130.1aece0680a18a95d51d6e1a5f83631412da37b87db65380c52052161354505ba
Model config RobertaConfig {
  "_name_or_path": "siebert/sentiment-roberta-large-english",
  "architectures": [
    "RobertaForSequenceClassification"
  ],
  "attention_probs_dropout_prob": 0.1,
  "bos_token_id": 0,
  "classifier_dr

...

storing https://huggingface.co/siebert/sentiment-roberta-large-english/resolve/main/vocab.json in cache at /root/.cache/huggingface/transformers/b522c6365937d6f39045d31ba715daafd39604f04b745f9d3d5cd622ecd74408.bfdcc444ff249bca1a95ca170ec350b442f81804d7df3a95a2252217574121d7
creating metadata file for /root/.cache/huggingface/transformers/b522c6365937d6f39045d31ba715daafd39604f04b745f9d3d5cd622ecd74408.bfdcc444ff249bca1a95ca170ec350b442f81804d7df3a95a2252217574121d7
https://huggingface.co/siebert/sentiment-roberta-large-english/resolve/main/merges.txt not found in cache or force_download set to True, downloading to /root/.cache/huggingface/transformers/tmpcb_c79kl


Downloading:   0%|          | 0.00/446k [00:00<?, ?B/s]

storing https://huggingface.co/siebert/sentiment-roberta-large-english/resolve/main/merges.txt in cache at /root/.cache/huggingface/transformers/a1fbb0cbc048b898b9adb0aa928b6bde50f393786ec91ffe195736820c42b02f.f5b91da9e34259b8f4d88dbc97c740667a0e8430b96314460cdb04e86d4fc435
creating metadata file for /root/.cache/huggingface/transformers/a1fbb0cbc048b898b9adb0aa928b6bde50f393786ec91ffe195736820c42b02f.f5b91da9e34259b8f4d88dbc97c740667a0e8430b96314460cdb04e86d4fc435
https://huggingface.co/siebert/sentiment-roberta-large-english/resolve/main/special_tokens_map.json not found in cache or force_download set to True, downloading to /root/.cache/huggingface/transformers/tmpdxnmwy9g


Downloading:   0%|          | 0.00/150 [00:00<?, ?B/s]

storing https://huggingface.co/siebert/sentiment-roberta-large-english/resolve/main/special_tokens_map.json in cache at /root/.cache/huggingface/transformers/e7bd01a8669e2d76258ba5ab711ba48da69b2dfc573c7b02566c0e73bd4583f4.0dc5b1041f62041ebbd23b1297f2f573769d5c97d8b7c28180ec86b8f6185aa8
creating metadata file for /root/.cache/huggingface/transformers/e7bd01a8669e2d76258ba5ab711ba48da69b2dfc573c7b02566c0e73bd4583f4.0dc5b1041f62041ebbd23b1297f2f573769d5c97d8b7c28180ec86b8f6185aa8
loading file https://huggingface.co/siebert/sentiment-roberta-large-english/resolve/main/vocab.json from cache at /root/.cache/huggingface/transformers/b522c6365937d6f39045d31ba715daafd39604f04b745f9d3d5cd622ecd74408.bfdcc444ff249bca1a95ca170ec350b442f81804d7df3a95a2252217574121d7
loading file https://huggingface.co/siebert/sentiment-roberta-large-english/resolve/main/merges.txt from cache at /root/.cache/huggingface/transformers/a1fbb0cbc048b898b9adb0aa928b6bde50f393786ec91ffe195736820c42b02f.f5b91da9e34259b8f4

In [65]:
task_to_keys = {
    "cola": ("sentence", None),
    "mnli": ("premise", "hypothesis"),
    "mnli-mm": ("premise", "hypothesis"),
    "mrpc": ("sentence1", "sentence2"),
    "qnli": ("question", "sentence"),
    "qqp": ("question1", "question2"),
    "rte": ("sentence1", "sentence2"),
    "sst2": ("sentence", None),
    "stsb": ("sentence1", "sentence2"),
    "wnli": ("sentence1", "sentence2"),
}


In [66]:
sentence1_key, sentence2_key = task_to_keys[task]
if sentence2_key is None:
    print(f"Sentence: {dataset['train'][0][sentence1_key]}")
else:
    print(f"Sentence 1: {dataset['train'][0][sentence1_key]}")
    print(f"Sentence 2: {dataset['train'][0][sentence2_key]}")

Sentence: hide new secretions from the parental units 


In [67]:
def preprocess_function(examples):
    if sentence2_key is None:
        return tokenizer(examples[sentence1_key], truncation=True)
    return tokenizer(examples[sentence1_key], examples[sentence2_key], truncation=True)

In [68]:
preprocess_function(dataset['train'][:5])

{'input_ids': [[0, 37265, 92, 3556, 2485, 31, 5, 20536, 2833, 1437, 2], [0, 10800, 5069, 117, 22094, 2156, 129, 6348, 3995, 821, 8299, 1437, 2], [0, 6025, 6138, 63, 3768, 8, 39906, 402, 1195, 2721, 59, 1050, 2574, 1437, 2], [0, 5593, 5069, 19223, 10028, 7, 1091, 5, 276, 1328, 1437, 2], [0, 261, 5, 2373, 13543, 12, 1116, 12, 627, 12, 1396, 11622, 43848, 5739, 5, 17504, 115, 31120, 1899, 62, 1437, 2]], 'attention_mask': [[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]]}

In [69]:
encoded_dataset = dataset.map(preprocess_function, batched=True)

  0%|          | 0/68 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

  0%|          | 0/2 [00:00<?, ?ba/s]

In [70]:
from transformers import AutoModelForSequenceClassification, TrainingArguments, Trainer

num_labels = 3 if task.startswith("mnli") else 1 if task=="stsb" else 2
model = AutoModelForSequenceClassification.from_pretrained(model_checkpoint, num_labels=num_labels)


loading configuration file https://huggingface.co/siebert/sentiment-roberta-large-english/resolve/main/config.json from cache at /root/.cache/huggingface/transformers/228e83e1ade2247aebc5f0725e330fa58dedee3d9eec36c9249f25084a946130.1aece0680a18a95d51d6e1a5f83631412da37b87db65380c52052161354505ba
Model config RobertaConfig {
  "_name_or_path": "siebert/sentiment-roberta-large-english",
  "architectures": [
    "RobertaForSequenceClassification"
  ],
  "attention_probs_dropout_prob": 0.1,
  "bos_token_id": 0,
  "classifier_dropout": null,
  "eos_token_id": 2,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 1024,
  "id2label": {
    "0": "NEGATIVE",
    "1": "POSITIVE"
  },
  "initializer_range": 0.02,
  "intermediate_size": 4096,
  "label2id": {
    "NEGATIVE": 0,
    "POSITIVE": 1
  },
  "layer_norm_eps": 1e-05,
  "max_position_embeddings": 514,
  "model_type": "roberta",
  "num_attention_heads": 16,
  "num_hidden_layers": 24,
  

Downloading:   0%|          | 0.00/1.32G [00:00<?, ?B/s]

storing https://huggingface.co/siebert/sentiment-roberta-large-english/resolve/main/pytorch_model.bin in cache at /root/.cache/huggingface/transformers/8fda88182b287ca423f06cae23d4c4701aaba057b65b3a691930de41e6537b3b.f135db3906a2e0c1f328a5920119f906cfb27b985cb0bfe7ef94434bdb31d031
creating metadata file for /root/.cache/huggingface/transformers/8fda88182b287ca423f06cae23d4c4701aaba057b65b3a691930de41e6537b3b.f135db3906a2e0c1f328a5920119f906cfb27b985cb0bfe7ef94434bdb31d031
loading weights file https://huggingface.co/siebert/sentiment-roberta-large-english/resolve/main/pytorch_model.bin from cache at /root/.cache/huggingface/transformers/8fda88182b287ca423f06cae23d4c4701aaba057b65b3a691930de41e6537b3b.f135db3906a2e0c1f328a5920119f906cfb27b985cb0bfe7ef94434bdb31d031
All model checkpoint weights were used when initializing RobertaForSequenceClassification.

All the weights of RobertaForSequenceClassification were initialized from the model checkpoint at siebert/sentiment-roberta-large-engl

In [78]:
metric_name = "pearson" if task == "stsb" else "matthews_correlation" if task == "cola" else "accuracy"
model_name = model_checkpoint.split("/")[-1]

args = TrainingArguments(
    f"{model_name}-finetuned-{task}",
    evaluation_strategy = "epoch",
    save_strategy = "epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    num_train_epochs=1,
    weight_decay=0.01,
    load_best_model_at_end=True,
    metric_for_best_model=metric_name,
    push_to_hub=True,
)


PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).


In [72]:
def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    if task != "stsb":
        predictions = np.argmax(predictions, axis=1)
    else:
        predictions = predictions[:, 0]
    return metric.compute(predictions=predictions, references=labels)

In [79]:
validation_key = "validation_mismatched" if task == "mnli-mm" else "validation_matched" if task == "mnli" else "validation"
trainer = Trainer(
    model,
    args,
    train_dataset=encoded_dataset["train"],
    eval_dataset=encoded_dataset[validation_key],
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)


/content/sentiment-roberta-large-english-finetuned-sst2 is already a clone of https://huggingface.co/yutoCHI/sentiment-roberta-large-english-finetuned-sst2. Make sure you pull the latest changes with `repo.git_pull()`.


In [80]:
trainer.train()

The following columns in the training set  don't have a corresponding argument in `RobertaForSequenceClassification.forward` and have been ignored: sentence, idx.
***** Running training *****
  Num examples = 67349
  Num Epochs = 1
  Instantaneous batch size per device = 16
  Total train batch size (w. parallel, distributed & accumulation) = 16
  Gradient Accumulation steps = 1
  Total optimization steps = 4210


Epoch,Training Loss,Validation Loss,Accuracy
1,0.186300,0.194301,0.952982


The following columns in the evaluation set  don't have a corresponding argument in `RobertaForSequenceClassification.forward` and have been ignored: sentence, idx.
***** Running Evaluation *****
  Num examples = 872
  Batch size = 16
Saving model checkpoint to sentiment-roberta-large-english-finetuned-sst2/checkpoint-4210
Configuration saved in sentiment-roberta-large-english-finetuned-sst2/checkpoint-4210/config.json
Model weights saved in sentiment-roberta-large-english-finetuned-sst2/checkpoint-4210/pytorch_model.bin
tokenizer config file saved in sentiment-roberta-large-english-finetuned-sst2/checkpoint-4210/tokenizer_config.json
Special tokens file saved in sentiment-roberta-large-english-finetuned-sst2/checkpoint-4210/special_tokens_map.json
tokenizer config file saved in sentiment-roberta-large-english-finetuned-sst2/tokenizer_config.json
Special tokens file saved in sentiment-roberta-large-english-finetuned-sst2/special_tokens_map.json


Training completed. Do not forget to sh

TrainOutput(global_step=4210, training_loss=0.22451777061770478, metrics={'train_runtime': 2183.9024, 'train_samples_per_second': 30.839, 'train_steps_per_second': 1.928, 'total_flos': 4405947603866172.0, 'train_loss': 0.22451777061770478, 'epoch': 1.0})

In [81]:
trainer.evaluate()

The following columns in the evaluation set  don't have a corresponding argument in `RobertaForSequenceClassification.forward` and have been ignored: sentence, idx.
***** Running Evaluation *****
  Num examples = 872
  Batch size = 16


{'epoch': 1.0,
 'eval_accuracy': 0.9529816513761468,
 'eval_loss': 0.19430142641067505,
 'eval_runtime': 7.4883,
 'eval_samples_per_second': 116.449,
 'eval_steps_per_second': 7.345}